In [1]:
import sys
import os
import torch
from pathlib import Path
import matplotlib.pyplot as plt
PROJECT_DIR = "../../../../OCT-Alzheimer-Project/"
sys.path.append(PROJECT_DIR)
print("appended",PROJECT_DIR)
from scripts import dataloader
from scripts import generators
from scripts.ddpm import ddpm_tools
from scripts.ddpm import trainers
from  generative.networks.nets import diffusion_model_unet as dmu
from generative.networks.nets import AutoencoderKL, DiffusionModelUNet, PatchDiscriminator
from generative.inferers import DiffusionInferer
from generative.networks.schedulers import DDPMScheduler, PNDMScheduler
from accelerate import notebook_launcher
import numpy as np
from torch.cuda.amp import autocast
import math 
import torch
import torch.nn as nn
import re

appended ../../../../OCT-Alzheimer-Project/
appended ../../../../OCT-Alzheimer-Project/


In [10]:
def training_script(devices,lr_schedule,checkpoint=None):
    if checkpoint:
        ckpt = str(re.findall(r"\d{4,}",checkpoint)[0])
    else:
        ckpt = checkpoint
#     devices = map(float, devices.strip('[]').split(','))
    print(devices)
    generator = generators.FundusGenerator(
        experiment_id = "ddpm_fundus_faf_AmyloidPET",
        workspace = Path(PROJECT_DIR),
        n_classes = 2,
        image_type = "generic_faf",
        condition = "AmyloidPET",
        old_expid =  ckpt
    )
    
    lr_schedule = True
    
    generator._initialise_data_loader(
        2,
        (256,256),
        10,
        num_workers = 8
    )
    scheduler = DDPMScheduler(num_train_timesteps=1000)    
    inferer = DiffusionInferer(scheduler)
    
    
    if devices[0] == "DDP":
        model = DiffusionModelUNet(
#             spatial_dims=2,
#             in_channels=1,
#             out_channels=1,
#             num_channels=(32,64,128),
#             attention_levels=(False, True, True),
#             num_res_blocks=1,
#             num_head_channels=(0,32,32),
#             with_conditioning=True,
#             cross_attention_dim=1,
            spatial_dims=2,
            in_channels=1,
            out_channels=1,
            num_channels=(32,64,128),
            attention_levels=(False, False, True),
            num_res_blocks=1,
            num_head_channels=(0,0,64),
            with_conditioning=True,
            cross_attention_dim=1,
        )
        
        optimizer = torch.optim.Adam(params=model.parameters(), lr= 2.5e-5)
        if lr_schedule:
            lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer,5,2,eta_min=1e-6)
        else:
            lr_scheduler=None
        
        trainer = trainers.AcceleratorTrainer(model=model,generator=generator,scheduler=scheduler,optimizer=optimizer,
                           inferer=inferer,n_epochs=10,lr_scheduler = lr_scheduler, ignore_blue=False,amp=True,
                                              checkpoint = checkpoint)        
    
    elif len(devices) == 1:
        model = DiffusionModelUNet(
#             spatial_dims=2,
#             in_channels=1,
#             out_channels=1,
#             num_channels=(32,64,128),
#             attention_levels=(False, True, True),
#             num_res_blocks=1,
#             num_head_channels=(0,32,32),
#             with_conditioning=True,
#             cross_attention_dim=1,
            spatial_dims=2,
            in_channels=1,
            out_channels=1,
            num_channels=(32,64,128),
            attention_levels=(False, False, True),
            num_res_blocks=1,
            num_head_channels=(0,0,64),
            with_conditioning=True,
            cross_attention_dim=1,
        )
        
        optimizer = torch.optim.Adam(params=model.parameters(), lr= 2.5e-5)
        if lr_schedule:
            lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer,5,2,eta_min=1e-6)
        else:
            lr_scheduler=None
        
        trainer = trainers.SingleGPUTrainer(model=model,generator=generator,scheduler=scheduler,optimizer=optimizer,
                           inferer=inferer,n_epochs=10,lr_scheduler = lr_scheduler, device = devices[0], 
                                            ignore_blue=False,checkpoint=checkpoint)

    else:
        model= trainers.ModelParallel(
#             spatial_dims=2,
#             in_channels=1,
#             out_channels=1,
#             num_channels=(32,64,128),
#             attention_levels=(False, True, True),
#             num_res_blocks=1,
#             num_head_channels=(0,32,32),
#             with_conditioning=True,
#             cross_attention_dim=1,
            spatial_dims=2,
            in_channels=1,
            out_channels=1,
            num_channels=(32,64,128),
            attention_levels=(False, False, True),
            num_res_blocks=1,
            num_head_channels=(0,0,64),
            with_conditioning=True,
            cross_attention_dim=1,
        )
        optimizer = torch.optim.Adam(params=model.parameters(), lr= 2.5e-5)
        if lr_schedule:
            lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer,5,2,eta_min=1e-6)
        else:
            lr_scheduler=None
            
        trainer = trainers.ParallelTrainer(model=model,generator=generator,scheduler=scheduler,optimizer=optimizer,
                           inferer=inferer,n_epochs=20,lr_scheduler = lr_scheduler, devices = devices, 
                                           ignore_blue=False,amp=True)
           
    

    trainer.train()


In [3]:
training_script(['cuda:0','cuda:1'],"/vol/tensusers4/islootweg/OCT-Alzheimer-Project/results/ddpm_fundus_faf_AmyloidPET/20231025111823/best_epoch")


gpu: cuda:0 -- 
 ['conv_in', 'time_embed', 'down_blocks']
gpu: cuda:1 -- 
 ['middle_block', 'up_blocks', 'out']
Training with model and parallel
Generator class arguments:
experiment_name: ddpm_fundus_faf_AmyloidPET
workspace: ../../../../OCT-Alzheimer-Project
image_type: generic_faf
condition: AmyloidPET
gray_scale: True
n_classes: 2
save_model: True
date_stamp: 20231025121051
exp_id: ddpm_fundus_faf_AmyloidPET_20231025121051
data_dir: ../../../../OCT-Alzheimer-Project/data


Epoch 0:  26%|█████▌               | 46/175 [00:14<00:42,  3.07it/s, loss=0.947]


KeyboardInterrupt: 

In [4]:
args = (["DDP"],True,None)
notebook_launcher(training_script,args,num_processes=1)

Launching training on one GPU.
Training with data parallel
Generator class arguments:
experiment_name: ddpm_fundus_faf_AmyloidPET
workspace: ../../../../OCT-Alzheimer-Project
image_type: generic_faf
condition: AmyloidPET
gray_scale: True
n_classes: 2
save_model: True
date_stamp: 20231025121408
exp_id: ddpm_fundus_faf_AmyloidPET_20231025121408
data_dir: ../../../../OCT-Alzheimer-Project/data


Epoch 0:   0%|                                          | 0/875 [00:04<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 GiB. GPU 0 has a total capacty of 14.58 GiB of which 1.15 GiB is free. Process 3199082 has 1.17 GiB memory in use. Including non-PyTorch memory, this process has 12.25 GiB memory in use. Of the allocated memory 11.88 GiB is allocated by PyTorch, and 239.50 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [11]:
training_script([1],False,None)

[1]
Training on one GPU:  cuda:1
Generator class arguments:
experiment_name: ddpm_fundus_faf_AmyloidPET
workspace: ../../../../OCT-Alzheimer-Project
image_type: generic_faf
condition: AmyloidPET
gray_scale: True
n_classes: 2
save_model: True
date_stamp: 20231025124213
exp_id: ddpm_fundus_faf_AmyloidPET_20231025124213
data_dir: ../../../../OCT-Alzheimer-Project/data


Epoch 0:   2%|▌                    | 21/875 [00:14<09:51,  1.44it/s, loss=0.949]


KeyboardInterrupt: 

In [ ]:
! accelerate launch --num_processes=1 --mixed_precision=fp16 2d_ddpm_faf-DDP.py None false

In [8]:
!nvidia-smi

Wed Oct 25 12:40:26 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.10              Driver Version: 535.86.10    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       On  | 00000000:03:00.0 Off |                    0 |
| N/A   47C    P0              27W /  70W |  12864MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
! kill -9 3199082 

/bin/bash: line 1: kill: (3199082) - Operation not permitted


In [ ]:
! accelerate launch --num_processes=2 --mixed_precision=fp16 2d_ddpm_faf-DDP.py None false

In [ ]:
df1 = pd.DataFrame({"str":[1,2],"n":["a","b"]})